# Lab 3.8.2: Classifiers—Weather Dataset
It supposedly concerns the weather conditions that are suitable for playing some unspecified game. There are four predictor variables: outlook, temperature, humidity, and wind. The outcome is whether to play (“yes,” “no,” “maybe”).<p>

Step 1: Build a classifier using logistic regression and then kNN, with each learning how various predictor variables could relate to the outcome.<p>
Step 2: Report the accuracies of your models.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
df = pd.read_csv('LAB 3.8.2-weather.csv')
print(df.columns)
# df.columns = ['']
display(df)

Index(['outlook', 'temperature', 'humidity', 'windy', 'play'], dtype='object')


,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,maybe
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,maybe
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,maybe


In [7]:
df['play_cat'] = df['play'].astype('category').cat.codes
df['windy_cat'] = df['windy'].astype('category').cat.codes
display(df)

,outlook,temperature,humidity,windy,play,play_cat,windy_cat
0,sunny,85,85,False,maybe,0,0
1,sunny,80,90,True,no,1,1
2,overcast,83,86,False,yes,2,0
3,rainy,70,96,False,maybe,0,0
4,rainy,68,80,False,yes,2,0
5,rainy,65,70,True,no,1,1
6,overcast,64,65,True,yes,2,1
7,sunny,72,95,False,no,1,0
8,sunny,69,70,False,yes,2,0
9,rainy,75,80,False,maybe,0,0


In [10]:
print(df['play_cat'].dtype)
print(df[['temperature', 'humidity', 'windy_cat']].corr())
print(df[['temperature', 'humidity', 'windy_cat']].nunique())


int8
             temperature  humidity  windy_cat
temperature     1.000000  0.315082  -0.328866
humidity        0.315082  1.000000  -0.201805
windy_cat      -0.328866 -0.201805   1.000000
temperature    12
humidity       10
windy_cat       2
dtype: int64


NOT ENOUGH data to split into test/train

In [27]:
# Feature matrix (X) and target vector (y)
X = df[['temperature', 'humidity', 'windy_cat']]  # Use reading, writing, and math as features
y = df['play_cat']  # 'program' is the target (categorical)

# Add a constant term (for intercept)
X = sm.add_constant(X)

# Fit the multinomial logistic regression model
model = sm.MNLogit(y, X)
result = model.fit()
print(result.summary())

# Predict on the test set
y_pred_prob = result.predict(X_test)  # Probabilities for each class

# Get the predicted class by choosing the class with the highest probability
y_pred = y_pred_prob.idxmax(axis=1)

# Calculate accuracy
accuracy_logit = accuracy_score(y_test, y_pred)

print(f"Accuracy of Logit Model predicting Program: {accuracy_logit:.2f}")

Optimization terminated successfully.
         Current function value: 0.641064
         Iterations 9
                          MNLogit Regression Results                          
Dep. Variable:               play_cat   No. Observations:                   14
Model:                        MNLogit   Df Residuals:                        6
Method:                           MLE   Df Model:                            6
Date:                Mon, 14 Apr 2025   Pseudo R-squ.:                  0.4059
Time:                        19:52:42   Log-Likelihood:                -8.9749
converged:                       True   LL-Null:                       -15.106
Covariance Type:            nonrobust   LLR p-value:                   0.05637
 play_cat=1       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           4.4158     13.280      0.333      0.739     -21.612      30.443
temperature    -0.1142    

In [24]:
# Split before scaling!
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale AFTER splitting
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit KNN on train set
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_scaled, y_train)

# Predict and evaluate
y_pred = knn.predict(X_test_scaled)
accuracy_knn = accuracy_score(y_test, y_pred)
print(f"Accuracy of KNN Model predicting Program: {accuracy_knn:.2f}")

Accuracy of KNN Model predicting Program: 0.00


In [25]:
print(f"Accuracy of the Logit Model {accuracy_logit} is = KNN {accuracy_knn}")
print("However, neither accuracy is that good; I would not rely on either of these models.")

Accuracy of the Logit Model 0.3333333333333333 is = KNN 0.0
However, neither accuracy is that good; I would not rely on either of these models.
